## Импорты

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import os
from tqdm import tqdm_notebook, trange, tqdm
import pickle
import numpy as np
from IPython.display import clear_output
import time
import string

# Предобработка

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# base_path = '/content/drive/My Drive/vkr'
base_path = '.'

- брать код из обычных репозиториев, а разбивку — из нормализованных
- сначала делаем выборку максимально без изворотов:
    - каждый файл нарезаем на части по n символов в каждом, если в последнем куске не n — выбрасываем
    - сохраняем выборку в файл pickle, пример: `[["import pickl",1], ...]` (потому что дальше идет "е", вероятно)
- преобразовать в удобный для модели формат:
    - вместо строк — массив чисел, где каждое число биективно соответствует символу
    
- модель учится предсказывать n+1 символ, если он eng letter, то это 1, иначе 0

In [4]:
class PreprocessDataset():
    def __init__(
        self,
        n,
        path_to_filenames,
        path_to_repo,
        objects_limit,
        check_if_english=True
    ):
        self.n = n
        self.total_count = 0
        self.pos_count = 0
        with open(path_to_filenames, 'r') as f:
            self.filenames = f.read().split('\n')
        self.path_to_repo = path_to_repo
        self.dataset_dictionary = set()
        self.objects_limit = objects_limit
        self.check_if_english = check_if_english
        self.allowed_letters = set(string.printable)

    def update_dataset_dictionary(self, code):
        self.dataset_dictionary.update(set(code))
        
    def is_english(self, code):
        for char in code:
            if char not in self.allowed_letters:
                return False
        return True

    def process_code(self, code):
        self.update_dataset_dictionary(code)
        chunks, chunk_size = len(code), self.n
        slices = [code[i:i+chunk_size] for i in range(0, chunks, chunk_size)]
        self.total_count += len(slices) - 1
        targets = []
        for i in range(len(slices) - 1):
            if slices[i+1][0].isalpha():
                self.pos_count += 1
                targets.append(True)
            else:
                targets.append(False)
        slices = slices[:-1]
        objects_from_code = list(zip(slices, targets))
        return objects_from_code
        
    def make_dataset(self):
        fidx = 0
        pbar = tqdm(total=self.objects_limit)
        objects_from_code = []
        while self.total_count < self.objects_limit:
            try:
                with open(
                    self.path_to_repo + self.filenames[fidx], 'r'
                ) as code_file:
                    code = code_file.read()
            except IsADirectoryError:
                fidx += 1
                continue
            fidx += 1
            if self.check_if_english and not self.is_english(code):
                continue
            objects_from_code += self.process_code(code)
            pbar.update(self.total_count)
        pbar.close()
#         print(objects_from_code)

        print('Number of positives samples is {} out of {} ({:.2f}%)'.format(
            self.pos_count,
            self.total_count,
            100. * self.pos_count / self.total_count
        ))
        
#         print(len(self.dataset_dictionary), self.dataset_dictionary)
        
        return objects_from_code


In [5]:
prep_dataset = PreprocessDataset(
    20,
    base_path + '/pycodesuggest_py_repos_normalised/test_files.txt',
    base_path + '/pycodesuggest_py_repos_normalised/',
    objects_limit=2000000
)
objects_from_code = prep_dataset.make_dataset()


11942663337it [00:12, 963811656.23it/s]                       

Number of positives samples is 1051233 out of 2000054 (52.56%)


In [6]:
objects_from_code[:10]

[('from MyQR.terminal i', True),
 ('from MyQR.myqr impor', True),
 ('t run\nimport os\n\n\nde', True),
 ('f function1739():\n  ', False),
 ('  import argparse\n  ', False),
 ('  var2519 = argparse', False),
 ('.ArgumentParser()\n  ', False),
 ('  var2519.add_argume', True),
 ("nt('Words', help=\n  ", False),
 ("      'The words to ", True)]

In [7]:
len(prep_dataset.dataset_dictionary)

98

In [8]:
prep_dataset.dataset_dictionary

{'\t',
 '\n',
 '\x0c',
 ' ',
 '!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '{',
 '|',
 '}',
 '~'}

In [9]:
with open(base_path + '/objects_from_code_test.pickle', 'wb') as f:
    pickle.dump(objects_from_code, f)
